# Bellman-Ford Algorithm

Recall: Single-Source Shortest Pths:
 - Input directed graph G with source vertex s and edge lengths Le for all edges. 
     - No parallel dges. Might as well throw away all parallels except for smallest length.
 - For every destination v in V, compute the length of a shortest s-v path. 
 - Previously solved using Dijkstra's
 
Dijkstra's:
 - O(mlogn) running time using heaps. m = # edges, n = # vertices
 - Problem is, need to have all positive edges else not always correct
     - e.g. if edges correspond to financial transactions
 - Issue is, also not very distributed (relevant for Internet routing). Dijkstra's very centralized. 

**Shortest Paths with Neg Paths and Cycles**

Question: How do we define shortest paths when G has a negative cycle?
 - Negative cycle is when sum of edge costs in cycle < 0
 
Solutions:
 - 1. Compute the shortest s -> v path, allow cycles. 
     - However, this does not work bc can keep finding shorter path by running on cycle. Shortest path may be -inf
 - 2. Compute shortest cycle-free s-v path. 
     - Issue is that this is "np complete." No known computationally efficient solution. NP-hard (No polynomial algorithm unless P = NP (will be explained more later))
 - 3. Assume that, for now, graph does not have negative cycles (allow neg edges still
     - Automatically checked in Bellman-Ford. Either finds shortest path or returns a negative cycle.
     - If G has no negative-cost cycles, for every v, there is shortest path with at most n - 1 edges from s to v.
         - Assump controls how many edges are necessary to ensure a shortest path
         - Visits at most n - 1 vertices, cannot visit a vertice more than once because that would be a cycle. If rip out cycle, get another path from s to v so total edge length has gone down (bc directed cycle must be nonnegative or smthn)

## Optimal Substructure

Goal: Find shoretst path from s to every v in V. Length of paths = sum of edge costs. 
 - Or, output a negative cycle as an excuse for failure to compute shortest path distance. (This shown later)
 - Consider for now actually outputting a shortest path. 
 
Informal Optimal Substructure Idea:
 - Difficult bc Graph's are not necessarily having an obvious ordering (except for path graphs)
 - However, can exploit sequential nature of paths. Subpath of a shortest path should itself be shortest. 
     - Not clear how to define "smaller" and "larger" subproblems
     - Key: Artificially restrict the number of edges in a path. 
         - Subproblem size corresponds with number of permitted edges

Optimal Substructure Formally:
 - Lemma: Let G = (V,E) be a directed graph with edge lengths Le and source vertex s. G may or may not have neg cycle
 - For every v in V, i {1, 2, 3,...} # of edges allowed in path from s to v
     - Let optimal solution be P = shortest s to v path with at most i edges. Allowed to use cycles (even negative) bc i limits # of cycles used. 
         - Case 1: If P has <= i - 1 edges, it is a shortest s - v path with <= i - 1 edges
             - True by obvious contradiction, P purported to be shortest within i edges
         - Case 2: If P has i edges with last hop (w,v), w being some other edge in path from s to v, then P' is a shortest s - w path with <= (i - 1) edges. 
             - If Q (some other s to w path <= i - 1 edges), then Q + (w,v) is shorter than P' + (w,v) which contradicts the optomality of P. Thus, Case 2 true. 
 - For destination v, there are (1 + in-deg(v)) candidates for optimal solution to subproblem. 
     - Answer depends on which destination v we are talkin about
     - in-degree = # of edges of input graph that have v as their head.
     - Case 1 is one candidate: given i and v, inherit optimal solution to destination v
     - Case 2 comprises a choice for each possible (w,v) edge. w uses at most i - 1 edges + (w,v) tacked on. 
             


## Basic Algorithm

Let Li,v = min length of a s-v path with <= i edges (with cycles allowed)

Recurrence:
 - defined as +inf if no s-v path exists within <= i edges
 - for every v in V, i(1,2,3....)
     - Liv = min(
         - Case 1: L(i-1)v
         - Case 2: min (for all w,v in edges) (L(i-1,v) + Lwv)
 
 - Correctness: Brute-force search from the only i + i-n-degree candidates by optimal substructure lemma

If no negative cycles:
 - suppose input graph G has no negative cycles
 - Know n - 1 edges are always enough to capture a shortest path between S and any v
     - For s to v with at least N edges, visits at least N + 1 vertices. So, path must visit some vertex twice and in btwn two consecutive visits of a given vertex that's in a directed cycle
 - Shortest paths do not have cycles (because removing a cycle only decreases length since no negative cycles)
 - Thus, with path, must have <= n - 1 edges
 - So, only need to consider values from i = 0 to n - 1; more than n-1 means not useful
     - Ofc, i = 0 will not evaluate unless s = v
 
Subproblems: Compute Liv for all i in (0, 1, 2...n-1).
 - Linear per problem which is very good

Bellman-Ford:
 - Let A = 2-D array (indexed by i and v)
 - Base Case: i = 0, A[0,s] = 0 (i.e. s = v); A[0,v] = +inf
 - for i = 1,2,3...n - 1:
     - for v in V:
         - A[i, v] = min(:
             - A[i-1, v]
             - min(A[i - 1, v] + Lwv) for all (w,v) edges
             
As discussed, if G has no negative cycle, then the algorithm is correct with answers A[n - 1, v]'s 

Runtime: O(mn)
 - Exactly O(n^2) subproblems, (n subproblems for n vertices, so n^2). But, work per loop is not constant
 - Spend more than constant time solving subproblem. Brute Force search through list of candidates that is superconstant. # of candidates for above, as said, is in-degree(v) aka each arc going to v. This is as large as m - 1. 
 - So, in general, worse than O(n^2). So, O(nm)
     - Could also be O(n^3) as a upper bound but not as sharp. Remember, m is between n-1 and nChoose2 (basically n^2). 
 - Consider work done by a single iteration of outer for loop
     - Sum of in-degrees(v) for all v in V; in total m
     - n loops of outer for loop, so total O(nm). 
 - EX: if add one edge, the in-degree of one vertex goes up by exactly one. 

Stopping Early:
 - Do not alway need n-1 iterations of outer for loop. 
 - Suppose for some j < n - 1, A[j,v] = A[j-1, v] for all vertices of v
     - In this case, next iterations are all useless. Can stop here. 
 
Note: More considerations for space optimization later. For example, the above rule applies for individual vertices of v too I think since each iteration only looks at (i-1) for comparison.

## Detecting Negative Cycles

Extend the algorithm. Question: What if the inpput graph G has a negative cycle? Want an algorithm to report this. 

Claim: Envision running outer for loop for one more (i.e. with i = n)
 - G has no negative-cost cycle iff in the extended algorithm, A[n-1, v] = A[n,v] for all v in V. 
 - As such, with an arbitrary input graph, can detect shortest paths OR negative cost cycle. This extra cycle makes it still O(n,m)
     - Note Edge Case: iif no path from s to every v, this not true. Imagine s not connected to anything, rest of graph is negative cycle. 
     - Modify: G has no negative-cost cycle reachable from s
     - Still detect by adding random vertex, extend an edge from that to every other vertex in G. Then run Bellman-Ford.
     
Proof:
 - Can begin with proving left or right side of above statement. 
 - Already proved taht Bellman-Ford is correct.
 - So, assume run Bellman-Ford for an extra iteration and non of subproblems change i.e. A[n-1, v] = A[n, v] for all v in V. (Assume finite so paths exist)
 - Let d(v) denote the common value of A[n - 1, v] and A[n, v]
 - Recall:
     - A[n,v] = min(A[n-1, v] or min A[n-1, w] + Lwv).
     - A[n-1, w] = d(w). A[n,v] = d(v). Thus, d(w) + Lwv >= d(v) for all edges w,v in E.
     - Equivalently: d(v) - d(w) <= Lwv
 - Consider arbitrary cycle C
     - sum(Lwv) for all wv in cycle >= sum(d(w) - d(v)) = 0 for w,v in Cycle. 
         - Note, each d value of vertex on a cycle is going to appear once with +1 coefficient and -1 coefficient. 
         - That's why sum(d(w) - d(v)) = 0. 

## Space Optimization

Current basic algorithm requires Theta(n^2) space. Space is dominated by 2D array, constant space per subproblem, n^2 subproblems. Remember, i (n-1) by n destinations. Can get away with linear sapce in an array baybee.

A[i,v] = minA[i - 1, v] or the min of A[i-1, w] + Lwvs
 - Note that only need the A[i - 1, v]s to compute the A[i,v]. Thus, can throw out all other subproblems, only need most recent info (as predicted above). 
 - Thus, only keeping track of O(n) to remember current and last round of subproblems. 
 - Outputs a linear output of things, so this is really good. Space is constant per computed statistic. 
 - Many other dynamic algorithms can do a similar algorithm probably (as we did with knapsack). 
 
Drawbacks?
 - Will not be able to run reconstruction because does not remember all of the subproblems by usual methods. Do differently

### Computing Predecessor Pointers

Idea: Comptue a second table B where B[i, v] = 2nd to last vertex (predecessor to v) on a shortest s -> v path with <= i edges (or Null if no path exists) for given i and v. Constant space per subproblem still bc i and v constant. 
 - Keeps track of "predecessor pointers"
 - Are there not O(n^2) subproblems? Am confuse. I guess for specific v, is O(n). 
 
Reconstruction: Assume the input graph G has no negative cycles and correctly compute the B[i, v]s. Then, tracing back predecessor pointers (B[n-1,v]'s) from v to s yields a shortest s - v path. With changes in i, traverses path. 
 - Comes from optimal substructure thing.
 
Computing Predecessor Pointers: 
 - B[0,v] = Null for all v in V. Base Case
 - Remember, A[i,v] either inherits from last round or A[i-1, w] + Lwv. B pretty much remembers w
     - Case 1: Last hop inherited:
         - B[i,v] = B[i-1, v]
     - Case 2: New hop
         - B[i,v] = w (vertex w that achieved the minimum)
 - Correctness: Computation of A[i,v] is brute-force search through 1 + in-deg(v) possible optimal solutions. B[i,v] is just caching the last hop of the winner. 
     - Can also be done with negative cost cycle detection. If B[n,v] improves from B[n-1, v], know negatie cost cycle exists. 
     - Check for cycle in predecessor pointers via DFS (below)
     - To reconstruct negative-cost cycle, use DFS to check for a cycle of predecessor pointers after each round (must be a negative cost cycle)